In [1]:
import matplotlib.pyplot as plt
import numpy as np
import keras

from sklearn.ensemble import VotingClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras.datasets import cifar100
from keras.wrappers.scikit_learn import KerasClassifier
from keras.datasets import mnist
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Activation, Dense, BatchNormalization, Dropout
from keras import optimizers

# import dataset, model and layers  
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import cv2
import random
import matplotlib.pyplot as plt
from scipy import misc
from __future__ import print_function
from tensorflow import keras
from tensorflow.keras import layers
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
#import splitfolders
from keras import Model
from keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import img_to_array, load_img
from keras.layers import ZeroPadding2D
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D,Dropout, Flatten, BatchNormalization, Concatenate, AveragePooling2D
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras.datasets import cifar100
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


## Load Dataset

In [2]:
batch_size = 32
num_classes = 100
epochs = 5
data_augmentation = True
num_predictions = 20

In [3]:
# example of loading the cifar10 dataset
from matplotlib import pyplot

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
print(y_train.shape)
print(y_test.shape)
#y_train = y_train.reshape(y_train.shape[0],)
#y_test = y_test.reshape(y_test.shape[0],)
#x_train = x_train.reshape(x_train.shape[0],(x_train.shape[1] * x_train.shape[2]*x_train.shape[3]))
#x_test = x_test.reshape(x_test.shape[0],(x_test.shape[1] * x_test.shape[2]*x_test.shape[3]))
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# plot first few images
#for i in range(9):
	# define subplot
	#pyplot.subplot(330 + 1 + i)
	# plot raw pixel data
	#pyplot.imshow(x_train[i])
# show the figure
#pyplot.show()

# Convert class vectors to binary class matrices.
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

#y_train.shape
#y_test.shape

(50000, 1)
(10000, 1)
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
def resnetmodel():
  # load model
  model1 = ResNet50(weights='imagenet', include_top=False,input_shape=(32,32,3),classes=num_classes)
  # summarize the model
  #model.summary()
  from keras import models
  from keras import layers
  from keras import optimizers

  # Create the model
  modelRes = models.Sequential()

  # Add the vgg convolutional base model
  modelRes.add(model1)

  # Add new layers
  modelRes.add(layers.Flatten())
  modelRes.add(layers.Dense(1024, activation='relu'))
  modelRes.add(layers.Dropout(0.5))
  modelRes.add(layers.Dense(num_classes, activation='softmax'))
  # Show a summary of the model. Check the number of trainable parameters
    # Let's train the model using RMSprop
      # initiate RMSprop optimizer
  opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
  modelRes.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
  return modelRes

In [5]:
 def mlpmodel(): 
  weight_decay = 4.7334492434534885e-09

  model = Sequential()
  model.add(Conv2D(32, (3,3), padding='same', strides = 1, activation='relu', 
                  kernel_initializer='he_uniform', 
                  kernel_regularizer=keras.regularizers.l2(weight_decay), 
                  ))

  model.add(BatchNormalization())
  model.add(Conv2D(32, (3,3), padding='same', strides = 1, activation='relu', 
                  kernel_initializer='he_uniform', 
                  kernel_regularizer=keras.regularizers.l2(weight_decay))) 
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.2))

  model.add(Conv2D(64, (3,3), padding='same', strides = 1, activation='relu', 
                  kernel_initializer='he_uniform', 
                  kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(BatchNormalization())
  model.add(Conv2D(64, (3,3), padding='same', strides = 1, activation='relu', 
                  kernel_initializer='he_uniform', 
                  kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.3))

  model.add(Conv2D(128, (3,3), padding='same', strides = 1, activation='relu', 
                  kernel_initializer='he_uniform', 
                  kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(BatchNormalization())
  model.add(Conv2D(128, (3,3), padding='same', strides = 1, activation='relu', 
                  kernel_initializer='he_uniform', 
                  kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.4))

  model.add(Conv2D(256, (3,3), padding='same', strides = 1, activation='relu', 
                  kernel_initializer='he_uniform', 
                  kernel_regularizer=keras.regularizers.l2(weight_decay)))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.6))

  model.add(Flatten())
  model.add(Dense(num_classes, activation='softmax'))

  adam = optimizers.Adam(lr = 0.0004190581840410445)
  model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

  return model
    

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    history = modelRes.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

Using real-time data augmentation.


In [7]:
# create 5 models to ensemble
from keras.wrappers.scikit_learn import KerasClassifier

model1 = KerasClassifier(build_fn = resnetmodel, epochs = 20)
model1._estimator_type = "classifier"
model2 = KerasClassifier(build_fn = mlpmodel, epochs = 20)
model2._estimator_type = "classifier"

In [8]:
ensemble_clf = VotingClassifier(estimators = [('model1', model1), ('model2', model2),], voting = 'soft')

In [ ]:
ensemble_clf.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/20
1563/1563 [==============================] - 112s 52ms/step - loss: 4.4000 - accuracy: 0.1119
Epoch 2/20
1563/1563 [==============================] - 80s 51ms/step - loss: 2.5488 - accuracy: 0.3617
Epoch 3/20
1563/1563 [==============================] - 79s 51ms/step - loss: 2.0439 - accuracy: 0.4669
Epoch 4/20
1563/1563 [==============================] - 79s 51ms/step - loss: 1.7196 - accuracy: 0.5405
Epoch 5/20
1563/1563 [==============================] - 80s 51ms/step - loss: 1.5100 - accuracy: 0.5925
Epoch 6/20
1563/1563 [==============================] - 79s 51ms/step - loss: 1.3165 - accuracy: 0.6393
Epoch 7/20
1563/1563 [==============================] - 79s 51ms/step - loss: 1.1671 - accuracy: 0.6765
Epoch 8/20
1205/1563 [======================>.......] - ETA: 18s - loss: 1.0497 - accuracy: 0.7095

In [ ]:
y_pred = ensemble_clf.predict(x_test)
print('Acc: ', accuracy_score(y_pred, y_test))